In [5]:
using RCall, Compat, Random, Distributions, Plots, DifferentialEquations

R"library(kdevine)"

function deriv_michaelis_menten(dydt, y, p, t)
    k_f = p[1]
    k_r = p[2]
    k_cat = p[3]
    dydt[1] = -k_f * y[1] * y[2] + k_r * y[3] + k_cat * y[3]
    dydt[2] = -k_f * y[1] * y[2] + k_r * y[3]
    dydt[3] = k_f * y[1] * y[2] - k_r * y[3] - k_cat * y[3]
    dydt[4] = k_cat * y[3]
end

function solve_michaelis_menten(initial, tspan, parameters)
    prob = ODEProblem(deriv_michaelis_menten, initial, tspan, parameters)
    sol = solve(prob)
    return sol
end

function f_solve(f_deriv, initial, tspan, parameters)
    prob = ODEProblem(f_deriv, initial, tspan, parameters)
    sol = solve(prob)
    return sol
end

function f_sample_solve_uniform(num_samples, population_sample, times_solve, tspan, f_solve, f_deriv, bounds, initial)
    n_params = size(bounds)[1]
    samples = zeros((num_samples, n_params))
    for i = 1:n_params
        samples[:, i] = rand(Uniform(bounds[i, 1], bounds[i, 2]), num_samples)
    end
    mOutputs = zeros((num_samples, length(population_sample)))
    for i in 1:num_samples
        sol = f_solve(f_deriv, initial, tspan, samples[i, :]);
        for j = 1:length(population_sample)
           mOutputs[i, j] = sol(times_solve[j], idxs=population_sample[j])
        end
    end
    return mOutputs
end

function f_estimate_kdevine(mOutputs)
    @rput mOutputs
    R"fit <- kdevine(mOutputs)"
end

function dkdevine(params)
    @rput params
    R"z <- dkdevine(params, fit)"
    @rget z
    return z
end

dkdevine (generic function with 1 method)

In [8]:
mBounds = [[0.2 15]; [0.2 2]; [0.5 3]];

tspan = (0.0, 10.0)

mOutputs = @time f_sample_solve_uniform(10000, [2, 1], [1, 2], tspan, f_solve, deriv_michaelis_menten, mBounds, [4.0; 8.0; 0.0; 0.0]);

@time f_estimate_kdevine(mOutputs)
@time dkdevine([2, 3])

  2.637255 seconds (12.82 M allocations: 1.094 GiB, 7.30% gc time)
 10.148413 seconds (81 allocations: 2.375 KiB)
  0.005798 seconds (97 allocations: 3.031 KiB)


0.028220048158615348